In [4]:
from utils import read_data, read_instances
from transformers import AutoModelForSeq2SeqLM, PreTrainedTokenizerFast, AutoTokenizer, AutoConfig
from transformers.models.bart.modeling_bart import BartForConditionalGeneration
import torch.nn.functional as F
import torch

# load data
file_path = "data/task1/train/eLife_train.jsonl"
instances = read_instances(file_path)

# tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")
# by default encoder-attention is `block_sparse` with num_random_blocks=3, block_size=64
# model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv") # 2.31G

Read data from :  data/task1/train/eLife_train.jsonl
The number of data:  4346


## Content Selection by Rouge Scores
select silient sentences

In [26]:
import numpy as np
from rouge import Rouge
from nltk import tokenize
rouge_pltrdy = Rouge()


def get_rouge2recall_scores_nopad(sentences, reference, oracle_type=None):
    # rouge_pltrdy is case sensitive
    reference = reference.lower()
    scores = [None for _ in range(len(sentences))]
    count_nonzero_rouge2recall = 0
    for i, sent in enumerate(sentences):
        sent = sent.lower()
        try:
            rouge_scores = rouge_pltrdy.get_scores(sent, reference)
            scores[i]  = rouge_scores[0]['rouge-2']['r'] # rouge2recall
        except ValueError:
            scores[i] = 0.0
        except RecursionError:
            scores[i] = 0.5 # just assign 0.5 as this sentence is simply too long
        if scores[i] > 0.0: count_nonzero_rouge2recall += 1
    # print('count_nonzero_rouge2recall=', count_nonzero_rouge2recall)
    scores = np.array(scores)
    N = len(scores)

    if oracle_type == 'padlead':
        biases = np.array([(N-i)*1e-12 for i in range(N)])
    elif oracle_type == 'padrand':
        biases = np.random.normal(scale=1e-10,size=(N,))
    else: # no pad 
        return np.array(scores)
    return np.array(scores) + biases

def compress_article(article):
    sentences = tokenize.sent_tokenize(article)
    # print(f'There are {len(sentences)} sentences.')
    reference = summaries[0]

    ## rank by ROUGH
    keep_idx = []
    scores = get_rouge2recall_scores_nopad( sentences, reference, oracle_type='padrand' )
    num_postive = sum(a > 0 for a in scores)
    rank = np.argsort(scores)[::-1][:num_postive] # only consider positive ones

    ## select high-ranked sentences
    keep_idx = []
    total_length = 0
    max_abssum_len = 1024-2
    for sent_i in rank:
        if total_length < max_abssum_len:
            sent = sentences[sent_i]
            total_length += len(bart_tokenizer.encode(sent)[1:-1]) # ignore <s> and </s>
            keep_idx.append(sent_i)
        else:
            break
    assert len(keep_idx) > 0
    ## if found nothing, selecting the top3 longest sentences
    # if len(keep_idx) == 0:
    #     sent_lengths = [len(tokenize.word_tokenize(ssent)) for ssent in sentences]
    #     keep_idx = np.argsort(sent_lengths)[::-1][:3].tolist()
    keep_idx = sorted(keep_idx) # back to original order
    filtered_sentences = [sentences[j] for j in keep_idx]
    filtered_input_text = " ".join(filtered_sentences)
    return filtered_input_text


In [19]:
compressed_articles = []
for i, article in enumerate(articles):
    print(i)
    filtered_input_text = compress_article(article)
    compressed_articles.append(compressed_articles)

There are 126 sentences.
count_nonzero_rouge2recall= 73


In [22]:
# with open(out_path, "w") as f:
#     f.write(filtered_input_text)
# print("write:", out_path)

'However , there is limited information on the timing and the relative magnitudes of maximum and minimum mortality , by local climate , age group , sex and medical cause of death . We used geo-coded mortality data and wavelets to analyse the seasonality of mortality by age group and sex from 1980 to 2016 in the USA and its subnational climatic regions . In adolescents and young adults , especially in males , death rates peaked in June/July and were lowest in December/January , driven by injury deaths . It is well-established that death rates vary throughout the year , and in temperate climates there tend to be more deaths in winter than in summer ( Campbell , 2017; Fowler et al . In a large country like the USA , which possesses distinct climate regions , the seasonality of mortality may vary geographically , due to geographical variations in mortality , localized weather patterns , and regional differences in adaptation measures such as heating , air conditioning and healthcare ( Davi

## Train BART

In [26]:
# load model
# tokenizer = PreTrainedTokenizerFast.from_pretrained("facebook/bart-base") # no <pad> token
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
bart = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
config = AutoConfig.from_pretrained("facebook/bart-base")

from torch.utils.data import Dataset, DataLoader

# create a dataset class for data loader
class SummarizationDataset(Dataset):
    def __init__(self, instances):
        self.instances = instances
        # sort the batch in the descending order of the number of tokens in the article
        instances.sort(key=lambda x: len(x['article']), reverse=True)

    def __len__(self):
        return len(self.instances)
    
    def __getitem__(self, idx):
        return self.instances[idx]['article'], self.instances[idx]['lay_summary']

def collate_fn(batch):
    """create a collate function for data loader. It will convert a batch (list) of texts with varying lengths into rectangular tensors"""
    # batch is a list of tuples (article, summary)
    articles = [item[0] for item in batch]
    summaries = [item[1] for item in batch]
    
    # tokenize the articles and summaries
    source_encodings = tokenizer(articles, padding="longest", truncation=True, return_tensors='pt')
    target_encodings = tokenizer(summaries, padding="longest", truncation=True, return_tensors='pt')

    batch = {key: source_encodings[key] for key in source_encodings}
    batch['labels'] = torch.tensor([ [-100 if id == tokenizer.pad_token_id else id for id in token_ids ] for token_ids in target_encodings['input_ids']])

    return batch

def sequence_cross_entropy_with_logits(logits, shifted_target_ids, target_mask=None):
    """Cross entropy that accepts logits and target sentence ids.
    Args:
        target_ids: [batch_size x sequence_length]   ground-truth predictions                         <s>   A   B C </s>  
                                                                                                       ^    ^   ^ ^ ^
        logits: [batch_size x sequence_length x vocab_size] logits generated from the input x         </s>  <s> A B C 
        
        shifted_target_mask: [batch_size x sequence_length] mask for the shifted target ids
    """
    # flatten
    logits_flat = logits.view(-1, logits.size(-1))
    targets_flatten = shifted_target_ids.view(-1)
    return F.cross_entropy(logits_flat, targets_flatten, target_mask)

dataset = SummarizationDataset(instances)


In [27]:
import math
from torch.optim.lr_scheduler import _LRScheduler
class PolynomialLR(_LRScheduler):
    """Decays the learning rate of each parameter group using a polynomial function
    in the given total_iters. When last_epoch=-1, sets initial lr as lr.
    Args:
        optimizer (Optimizer): Wrapped optimizer.
        total_iters (int): The number of steps that the scheduler decays the learning rate. Default: 5.
        power (int): The power of the polynomial. Default: 1.0.
        verbose (bool): If ``True``, prints a message to stdout for
            each update. Default: ``False``.
    Example:
        >>> # xdoctest: +SKIP("undefined vars")
        >>> # Assuming optimizer uses lr = 0.001 for all groups
        >>> # lr = 0.001     if epoch == 0
        >>> # lr = 0.00075   if epoch == 1
        >>> # lr = 0.00050   if epoch == 2
        >>> # lr = 0.00025   if epoch == 3
        >>> # lr = 0.0       if epoch >= 4
        >>> scheduler = PolynomialLR(self.opt, total_iters=4, power=1.0)
        >>> for epoch in range(100):
        >>>     train(...)
        >>>     validate(...)
        >>>     scheduler.step()
    """
    def __init__(self, optimizer, total_iters=5, power=1.0, last_epoch=-1, verbose=False):
        self.total_iters = total_iters
        self.power = power
        super().__init__(optimizer, last_epoch, verbose)

    def get_lr(self):
        if not self._get_lr_called_within_step:
            warnings.warn("To get the last learning rate computed by the scheduler, "
                          "please use `get_last_lr()`.", UserWarning)

        if self.last_epoch == 0 or self.last_epoch > self.total_iters:
            return [group["lr"] for group in self.optimizer.param_groups]

        decay_factor = ((1.0 - self.last_epoch / self.total_iters) / (1.0 - (self.last_epoch - 1) / self.total_iters)) ** self.power
        return [group["lr"] * decay_factor for group in self.optimizer.param_groups]

    def _get_closed_form_lr(self):
        return [
            (
                base_lr * (1.0 - min(self.total_iters, self.last_epoch) / self.total_iters) ** self.power
            )
            for base_lr in self.base_lrs
        ]


In [28]:
from tqdm import tqdm
from transformers import AdamW
import wandb
# import lazy_groups_of
from allennlp.common.util import lazy_groups_of

wandb.init(project="bart-summarization", name="bart-base")

    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bart = bart.to(device)
dataloader = DataLoader(dataset, batch_size=2, collate_fn=collate_fn)

batch_generator = iter(dataloader)
num_accumulation_steps = 4
num_training_steps = math.ceil(len(dataloader) / num_accumulation_steps)
batch_group_generator = lazy_groups_of(batch_generator, group_size=num_accumulation_steps)

optimizer = AdamW(bart.parameters(), lr=3e-5, betas=(0.9, 0.999), eps=1e-8, correct_bias=True)
lr_sceduler = PolynomialLR(optimizer, total_iters=num_training_steps, power=1.0)
for batch_group in tqdm(batch_group_generator):
    torch.cuda.empty_cache() 
    optimizer.zero_grad()
    batch_loss = 0
    for batch in batch_group:
        
        batch = {key: batch[key].to(device) for key in batch}

        # decoder input by shifting the labels to the right as the input: <s> A B C </s> -> </s> <s> A B C 
        # wrong ones: shift the labels to the right: <s> A B C </s> -> <s> A B C 
        # decoder_input_ids=batch['labels'][:, :-1].contiguous()
        # decoder_attention_mask=target_mask[:, :-1].contiguous()
        decoder_input_ids = batch['labels'].new_zeros(batch['labels'].shape)
        decoder_input_ids[:, 1:] = batch['labels'][:, :-1].clone().type(torch.LongTensor).contiguous()
        decoder_input_ids[:, 0] = bart.config.decoder_start_token_id
        decoder_input_ids.masked_fill_(decoder_input_ids == -100, bart.config.pad_token_id)
        outputs = bart(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], decoder_input_ids=decoder_input_ids, return_dict=True, use_cache=False,)
        
        loss = sequence_cross_entropy_with_logits(outputs.logits, batch['labels'].contiguous()) # target_mask.type(torch.LongTensor).contiguous()
        loss.backward()
        batch_loss += loss.item()
    
    optimizer.step()
    lr_sceduler.step()
    batch_loss /= num_accumulation_steps
    wandb.log({"loss": batch_loss})
    
wandb.finish()



loss,2.28172
_runtime,712
_timestamp,1681110647
_step,299


loss,▁▂▁▃▃▅▅▂▄▄▇▅▃▅▃▃▃▄▃▃▃▄▇▆▅▅▂▅▆▄▆▅▆▄▃█▃▂▅▄
_runtime,▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
_timestamp,▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: wandb version 0.14.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2023-04-10 17:12:35.757221: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-10 17:12:36.030167: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-10 17:12:36.831411: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory


1it [00:01,  1.65s/it]

...

Before scheduler: lr =  1.1029411764705871e-07
After scheduler: lr =  5.5147058823529354e-08


544it [17:50,  1.97s/it]

Before scheduler: lr =  5.5147058823529354e-08
After scheduler: lr =  0.0


loss,0.69175
_runtime,1076
_timestamp,1681111829
_step,543


loss,█▇▇▇▆▇▆▇▇▆▇▆▇▆▆▆▆▆▆▆▆▆▆▆▆▇▆▆▆▆▆▆▆▆▇▆▆▆▆▁
_runtime,▁▁▁▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


In [28]:
print(outputs.logits.shape)
print(batch['labels'].shape)

torch.Size([2, 673, 50265])
torch.Size([2, 673])


In [5]:
# iterate over epochs
epochs = 10
for epoch in range(epochs):
    pass

tensor(3.5736, device='cuda:0', grad_fn=<NllLossBackward>)